In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# first strategy: Simple MA 
# when the asset's open price is below the its SMA, and the volume is above the its SMA it's a buying signal, and vice versa for selling.

In [16]:
# load eth_daily.csv
df = pd.read_csv('/home/qian/qian/CryptoTrade/data/eth_daily.csv')
print(df.head())

                snapped_at      open   market_cap  total_volume
0  2015-08-07 00:00:00 UTC  2.831620         0.00  9.062200e+04
1  2015-08-08 00:00:00 UTC  1.330750  80339475.00  3.680700e+05
2  2015-08-10 00:00:00 UTC  0.687586  41556309.21  4.004641e+05
3  2015-08-11 00:00:00 UTC  1.067379  64539006.31  1.518998e+06
4  2015-08-12 00:00:00 UTC  1.256613  76013261.05  2.073893e+06


In [17]:
sma_periods = [5, 10, 15, 20, 30]
for period in sma_periods:
    df[f'SMA_{period}'] = df['open'].rolling(window=period).mean()

In [19]:
# print the df head now
print(df.head())

                snapped_at      open   market_cap  total_volume    SMA_5  \
0  2015-08-07 00:00:00 UTC  2.831620         0.00  9.062200e+04      NaN   
1  2015-08-08 00:00:00 UTC  1.330750  80339475.00  3.680700e+05      NaN   
2  2015-08-10 00:00:00 UTC  0.687586  41556309.21  4.004641e+05      NaN   
3  2015-08-11 00:00:00 UTC  1.067379  64539006.31  1.518998e+06      NaN   
4  2015-08-12 00:00:00 UTC  1.256613  76013261.05  2.073893e+06  1.43479   

   SMA_10  SMA_15  SMA_20  SMA_30  
0     NaN     NaN     NaN     NaN  
1     NaN     NaN     NaN     NaN  
2     NaN     NaN     NaN     NaN  
3     NaN     NaN     NaN     NaN  
4     NaN     NaN     NaN     NaN  


In [20]:
# print df tail
print(df.tail())

                   snapped_at         open    market_cap  total_volume  \
3116  2024-02-18 00:00:00 UTC  2788.561332  3.350000e+11  1.835907e+10   
3117  2024-02-19 00:00:00 UTC  2874.945276  3.450000e+11  2.408201e+10   
3118  2024-02-20 00:00:00 UTC  2943.723085  3.540000e+11  1.645019e+10   
3119  2024-02-21 00:00:00 UTC  3015.438952  3.620000e+11  2.250996e+10   
3120  2024-02-22 00:00:00 UTC  2977.082017  3.580000e+11  2.096431e+10   

            SMA_5       SMA_10       SMA_15       SMA_20       SMA_30  
3116  2768.932717  2641.604244  2539.756446  2482.329949  2428.703562  
3117  2815.614552  2687.001916  2578.438402  2510.347476  2441.547803  
3118  2848.284574  2732.589134  2622.014309  2540.333561  2457.329785  
3119  2886.008070  2784.183811  2669.798912  2576.981215  2476.014099  
3120  2919.950132  2831.384332  2710.086987  2610.748451  2498.330599  


In [21]:
# then filter the data from 2024-01-01 to 2024-01-31
df = df[df['snapped_at'] >= '2024-01-01']
print(len(df))
df = df[df['snapped_at'] <= '2024-02-01']
print(len(df))

53
31


In [22]:
# Initial variables
initial_capital = 1_000_000
capital = initial_capital
portfolio = 0  # Tracks the quantity of ETH held
previous_signals = {period: None for period in sma_periods}  # Track previous day signals to determine change

In [23]:
# print df head
print(df.head())

                   snapped_at         open    market_cap  total_volume  \
3068  2024-01-01 00:00:00 UTC  2279.941009  2.740000e+11  6.715146e+09   
3069  2024-01-02 00:00:00 UTC  2350.026286  2.820000e+11  9.173151e+09   
3070  2024-01-03 00:00:00 UTC  2360.782120  2.830000e+11  2.059026e+10   
3071  2024-01-04 00:00:00 UTC  2211.170828  2.660000e+11  3.445178e+10   
3072  2024-01-05 00:00:00 UTC  2270.977439  2.730000e+11  1.834110e+10   

            SMA_5       SMA_10       SMA_15       SMA_20       SMA_30  
3068  2319.168636  2300.326775  2268.745644  2262.887693  2267.087733  
3069  2313.819643  2302.624507  2278.778055  2270.372322  2273.245346  
3070  2316.923926  2307.607153  2288.368262  2275.367310  2278.914615  
3071  2299.252850  2302.233121  2290.657023  2270.052598  2277.854925  
3072  2294.579536  2301.967604  2295.494092  2272.603889  2277.084334  


In [24]:
# print df tail now
print(df.tail())

                   snapped_at         open    market_cap  total_volume  \
3094  2024-01-27 00:00:00 UTC  2268.870732  2.720000e+11  1.073704e+10   
3095  2024-01-28 00:00:00 UTC  2267.907852  2.730000e+11  4.896940e+09   
3096  2024-01-29 00:00:00 UTC  2257.054465  2.710000e+11  6.994794e+09   
3097  2024-01-30 00:00:00 UTC  2314.594728  2.780000e+11  8.669925e+09   
3098  2024-01-31 00:00:00 UTC  2344.001400  2.810000e+11  1.112645e+10   

            SMA_5       SMA_10       SMA_15       SMA_20       SMA_30  
3094  2254.950870  2368.520334  2424.557990  2423.422070  2379.635595  
3095  2247.015040  2342.478240  2407.451734  2425.809884  2377.057167  
3096  2250.037378  2321.450789  2386.193716  2422.079591  2375.641442  
3097  2265.608461  2303.948455  2375.304200  2420.751296  2376.316466  
3098  2290.485835  2291.322234  2363.882961  2408.512908  2378.451813  


In [25]:
# store the df now to a new csv file
df.to_csv('/home/qian/qian/CryptoTrade/data/eth_daily_with_sma.csv', index=False)

In [28]:
import pandas as pd

# Assuming the CSV file is named 'eth_price.csv' and located in your current working directory
file_path = '/home/qian/qian/CryptoTrade/data/eth_daily_with_sma.csv'
df = pd.read_csv(file_path)

# Initialize initial capital and other trading parameters
initial_capital = 1_000_000
sma_periods = [5, 10, 15, 20, 30]  # The SMA periods you have in your CSV
results = {}

def simulate_sma_strategy(df, sma_column):
    """
    Simulate trading strategy based on a single SMA column.
    
    :param df: DataFrame containing the asset and SMA data.
    :param sma_column: The column name of the SMA to base the trading signals on.
    :return: Final net worth after executing the trading strategy.
    """
    capital = initial_capital
    portfolio = 0  # Tracks the quantity of ETH held
    previous_signal = None  # Track the previous day signal
    
    # Iterate through each row in the DataFrame to simulate trading
    for index, row in df.iterrows():
        current_signal = None
        if row['open'] > row[sma_column]:
            current_signal = 'buy'
        elif row['open'] < row[sma_column]:
            current_signal = 'sell'
        
        # Execute trades based on signal changes
        if current_signal != previous_signal and current_signal is not None:
            trade_amount = capital * 0.2  # Use 20% of capital for each trade

            if current_signal == 'buy':
                # Buy ETH with the specified trade amount
                eth_bought = trade_amount / row['open']
                portfolio += eth_bought
                capital -= trade_amount
            elif current_signal == 'sell' and portfolio > 0:
                # Sell a portion of ETH holdings
                eth_sold = portfolio * 0.5 # Sell 20% of holdings
                capital += eth_sold * row['open']
                portfolio -= eth_sold

        previous_signal = current_signal

    # Calculate final net worth
    final_net_worth = capital + (portfolio * df.iloc[-1]['open'])
    return final_net_worth

# Run simulations for each SMA period and store results
for period in sma_periods:
    sma_column = f'SMA_{period}'
    net_worth = simulate_sma_strategy(df, sma_column)
    results[sma_column] = net_worth

# Display results
for sma, net_worth in results.items():
    print(f"{sma} Final Net Worth: ${net_worth:.2f}")
    print(f"{sma} Return: {((net_worth / initial_capital) - 1) * 100:.2f}%\n")


SMA_5 Final Net Worth: $1003672.15
SMA_5 Return: 0.37%

SMA_10 Final Net Worth: $1004325.73
SMA_10 Return: 0.43%

SMA_15 Final Net Worth: $1007441.81
SMA_15 Return: 0.74%

SMA_20 Final Net Worth: $998543.51
SMA_20 Return: -0.15%

SMA_30 Final Net Worth: $998543.51
SMA_30 Return: -0.15%



In [32]:
import pandas as pd

# Load the data
file_path = '/home/qian/qian/CryptoTrade/data/eth_daily_with_sma.csv'
df = pd.read_csv(file_path)

initial_capital = 1_000_000
sma_periods = [5, 10, 15, 20, 30]
results = []

def simulate_sma_strategy(df, sma_column, portfolio_ratio):
    capital = initial_capital
    portfolio = 0  # in ETH
    previous_signal = None

    for index, row in df.iterrows():
        current_signal = 'hold'
        if row['open'] > row[sma_column]:
            current_signal = 'buy'
        elif row['open'] < row[sma_column]:
            current_signal = 'sell'
        
        if current_signal != previous_signal:
            if current_signal == 'buy' and capital > 0:
                eth_bought = (capital * portfolio_ratio) / row['open']
                portfolio += eth_bought
                capital -= eth_bought * row['open']
            elif current_signal == 'sell' and portfolio > 0:
                eth_sold = portfolio * portfolio_ratio
                capital += eth_sold * row['open']
                portfolio -= eth_sold

        previous_signal = current_signal

    final_net_worth = capital + portfolio * df.iloc[-1]['open']
    return final_net_worth

# Portfolio ratio to adjust how much of the portfolio is traded
portfolio_ratio = 0.8  # Adjust this value as needed

# Run simulations
for period in sma_periods:
    sma_column = f'SMA_{period}'
    net_worth = simulate_sma_strategy(df, sma_column, portfolio_ratio)
    return_percentage = ((net_worth / initial_capital) - 1) * 100
    results.append({
        'SMA Period': period,
        'Final Net Worth': net_worth,
        'Return Percentage': return_percentage
    })

# Convert results to DataFrame and save to CSV
results_df = pd.DataFrame(results)
results_csv_path = 'sma_strategy_results.csv'
results_df.to_csv(results_csv_path, index=False)

print(f"Results saved to {results_csv_path}")


Results saved to sma_strategy_results.csv


In [40]:
# second strategy: MACD
# MACD = 12-day EMA - 26-day EMA
# Signal Line = 9-day EMA of MACD
# When MACD crosses above the signal line, it's a buying signal, and vice versa for selling.

# load the data
file_path = '/home/qian/qian/CryptoTrade/data/eth_daily.csv'
df = pd.read_csv(file_path)


# Calculate the MACD and Signal Line
df['EMA_12'] = df['open'].ewm(span=12, adjust=False).mean()
df['EMA_26'] = df['open'].ewm(span=26, adjust=False).mean()

df['MACD'] = df['EMA_12'] - df['EMA_26']

df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

# Print the head of the DataFrame
print(df.head())


                snapped_at      open   market_cap  total_volume    EMA_12  \
0  2015-08-07 00:00:00 UTC  2.831620         0.00  9.062200e+04  2.831620   
1  2015-08-08 00:00:00 UTC  1.330750  80339475.00  3.680700e+05  2.600717   
2  2015-08-10 00:00:00 UTC  0.687586  41556309.21  4.004641e+05  2.306389   
3  2015-08-11 00:00:00 UTC  1.067379  64539006.31  1.518998e+06  2.115772   
4  2015-08-12 00:00:00 UTC  1.256613  76013261.05  2.073893e+06  1.983594   

     EMA_26      MACD  Signal_Line  
0  2.831620  0.000000     0.000000  
1  2.720444 -0.119728    -0.023946  
2  2.569862 -0.263473    -0.071851  
3  2.458567 -0.342795    -0.126040  
4  2.369534 -0.385940    -0.178020  


In [56]:
# then apply the strategy on the date from 2024-01-01 to 2024-01-31
df = df[df['snapped_at'] >= '2024-01-01']
df = df[df['snapped_at'] <= '2024-02-01']


# Initialize initial capital and other trading parameters
initial_capital = 1_000_000

capital = initial_capital

portfolio = 0  # Tracks the quantity of ETH held


previous_signal = None  # Track the previous day signal

# Iterate through each row in the DataFrame to simulate trading
for index, row in df.iterrows():
    current_signal = None
    if row['MACD'] < row['Signal_Line']:
        current_signal = 'buy'
    elif row['MACD'] > row['Signal_Line']:
        current_signal = 'sell'
    
    # Execute trades based on signal changes
    if current_signal != previous_signal and current_signal is not None:
        trade_amount = capital * 1  # Use 100% of capital for each trade

        if current_signal == 'buy':
            # Buy ETH with the specified trade amount
            eth_bought = trade_amount / row['open']
            portfolio += eth_bought
            capital -= trade_amount
        elif current_signal == 'sell' and portfolio > 0:
            # Sell a portion of ETH holdings
            eth_sold = portfolio * 1 # Sell 100% of holdings
            capital += eth_sold * row['open']
            portfolio -= eth_sold

    previous_signal = current_signal
    
    
# Calculate final net worth
final_net_worth = capital + (portfolio * df.iloc[-1]['open'])

# store the final net worth to a csv file

print(f"Final Net Worth: ${final_net_worth:.2f}")

Final Net Worth: $1150317.94


In [57]:
# third strategy: short and long strategy - If the short period SMA is below the long period SMA, it means that the trend is going down, so it's a sell signal, it's also known as the death cross.
# Otherwise, the trend is shiftting up, and it's a buy signal, it's also called the golden cross.

# load the data 
file_path = '/home/qian/qian/CryptoTrade/data/eth_daily_with_sma.csv'

df = pd.read_csv(file_path)

In [58]:
print(df.head())

                snapped_at         open    market_cap  total_volume  \
0  2024-01-01 00:00:00 UTC  2279.941009  2.740000e+11  6.715146e+09   
1  2024-01-02 00:00:00 UTC  2350.026286  2.820000e+11  9.173151e+09   
2  2024-01-03 00:00:00 UTC  2360.782120  2.830000e+11  2.059026e+10   
3  2024-01-04 00:00:00 UTC  2211.170828  2.660000e+11  3.445178e+10   
4  2024-01-05 00:00:00 UTC  2270.977439  2.730000e+11  1.834110e+10   

         SMA_5       SMA_10       SMA_15       SMA_20       SMA_30  
0  2319.168636  2300.326775  2268.745644  2262.887693  2267.087733  
1  2313.819643  2302.624507  2278.778055  2270.372322  2273.245346  
2  2316.923926  2307.607153  2288.368262  2275.367310  2278.914615  
3  2299.252850  2302.233121  2290.657023  2270.052598  2277.854925  
4  2294.579536  2301.967604  2295.494092  2272.603889  2277.084334  


In [62]:
# if sma_5 is below sma_20, it's a sell signal, and vice versa for buying
# Initialize initial capital and other trading parameters

initial_capital = 1_000_000
capital = initial_capital

portfolio = 0  # Tracks the quantity of ETH held

previous_signal = None  # Track the previous day signal

# Iterate through each row in the DataFrame to simulate trading

for index, row in df.iterrows():
    current_signal = None
    if row['SMA_5'] < row['SMA_20']:
        current_signal = 'buy'
    elif row['SMA_5'] > row['SMA_20']:
        current_signal = 'sell'
    
    # Execute trades based on signal changes
    if current_signal != previous_signal and current_signal is not None:
        trade_amount = capital * 1  # Use 100% of capital for each trade

        if current_signal == 'buy':
            # Buy ETH with the specified trade amount
            eth_bought = trade_amount / row['open']
            portfolio += eth_bought
            capital -= trade_amount
        elif current_signal == 'sell' and portfolio > 0:
            # Sell a portion of ETH holdings
            eth_sold = portfolio * 1 # Sell 100% of holdings
            capital += eth_sold * row['open']
            portfolio -= eth_sold

    previous_signal = current_signal
    
    
# Calculate final net worth
final_net_worth = capital + (portfolio * df.iloc[-1]['open'])

print(f"Final Net Worth: ${final_net_worth:.2f}")

Final Net Worth: $1207496.36
